In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# import librairies
import pickle 
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import pandas

In [0]:
from google.colab import files
# dbfile=files.download('/content/vehicleimg')
db = {} 
# Its important to use binary mode 
dbfile = open('/content/drive/My Drive/vehicleimg', 'rb') 
new_dict = pickle.load(dbfile)          
print(new_dict.shape)        # (13785, 186, 250, 3)
dbfile.close() 

(13785, 186, 250, 3)


In [0]:
db = {} 
# Its important to use binary mode 
dbfile = open('/content/vehicletrgt', 'rb') 
labels = pickle.load(dbfile)          
print(labels.shape)          # (13785)
dbfile.close() 

(13785,)


In [0]:
# have to transforme the labels in porcentages
# there are 3 classes: 2, 3 and 4
labels_porcentages=np.zeros((len(labels),3))
for i in range(len(labels)):
  labels_porcentages[i][labels[i]-2]=1

In [0]:
import keras
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [0]:
for i in range(2000,4000): 
	it=datagen.flow(new_dict[i:i+1],  batch_size=2)
	batch = it.next()
	# convert to unsigned integers for viewing
	image = batch[:2].astype('uint8')
	new_dict=np.concatenate((new_dict,image),axis=0)
	labels_porcentages=np.concatenate((labels_porcentages,labels_porcentages[i:i+1]),axis=0)

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
# CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout
#create model
model = Sequential()#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(186,250,3)))
model.add(Dropout(0.25, noise_shape=None, seed=None))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Dropout(0.25, noise_shape=None, seed=None))
model.add(Flatten())
model.add(Dense(40, activation='softmax'))
model.add(Dropout(0.25, noise_shape=None, seed=None))
model.add(Dense(20, activation='softmax'))
model.add(Dropout(0.25, noise_shape=None, seed=None))
model.add(Dense(3, activation='softmax'))

In [0]:
#compile model using accuracy to measure model performance
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# preprocessing
import skimage.exposure
histo=[0]*5000
histo[:2000]=skimage.exposure.equalize_hist(new_dict[:2000])
histo[2000:4000]=skimage.exposure.equalize_hist(new_dict[7000:9000])
histo[4000:]=skimage.exposure.equalize_hist(new_dict[11000:12000])
l=np.stack(histo, axis=0 )
# can't do it in one time because else it crash

In [0]:
from sklearn.model_selection import train_test_split
y=np.concatenate((np.concatenate((labels_porcentages[:2000],labels_porcentages[7000:9000]),axis=0),labels_porcentages[11000:12000]),axis=0)
X_train, X_test, y_train, y_test = train_test_split(l, y, test_size=0.1, random_state=42)

In [27]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

Train on 4500 samples, validate on 500 samples
Epoch 1/3
4500/4500 [==============================] - 19s 4ms/step - loss: 0.7618 - acc: 0.6749 - val_loss: 0.5506 - val_acc: 0.8260
Epoch 2/3
4500/4500 [==============================] - 19s 4ms/step - loss: 0.7438 - acc: 0.6818 - val_loss: 0.5277 - val_acc: 0.8260
Epoch 3/3
4500/4500 [==============================] - 19s 4ms/step - loss: 0.7240 - acc: 0.6698 - val_loss: 0.5091 - val_acc: 0.8240


In [0]:
#predict first 4 images in the test set
model.predict(X_test[:20])

In [0]:
#actual results for first 4 images in test set
y_test[:20]